In [2]:
import sys
import pandas as pd

sys.path.append("../")
from pathlib import Path

In [24]:
# Cell 1: Load meeting data and extract video URLs

import json
import asyncio
import os
from pathlib import Path
from typing import List, Dict, Any
import pandas as pd
# Import the get_meetings function from our meetings module
from src.meetings import get_meetings
from src.models.meeting import Meeting
# Path to the meetings.jsonl file
jsonl_path = Path("../data/meetings.jsonl")
meetings = pd.read_json(jsonl_path, lines=True)
# Display the full meeting information without truncation
pd.set_option('display.max_colwidth', None)
meetings.iloc[2]


meeting                                                       Regular Council Meeting
date                                                        2025-02-26T17:00:00-06:00
date_display                                              February 26, 2025 - 5:00 PM
duration                                                                      00h 38m
agenda          https://tulsa-ok.granicus.com/AgendaViewer.php?view_id=4&clip_id=6399
video            https://tulsa-ok.granicus.com/MediaPlayer.php?view_id=4&clip_id=6399
Name: 2, dtype: object

In [25]:
# Cell to fetch and display the Granicus video player section using selectolax
import requests
from IPython.display import HTML, display
from selectolax.parser import HTMLParser

# URL for the Granicus video player page
granicus_url = "https://tulsa-ok.granicus.com/MediaPlayer.php?view_id=4&clip_id=6399"

# Fetch the HTML content
response = requests.get(granicus_url)
html_content = response.text

# Parse the HTML with selectolax
parser = HTMLParser(html_content)

# Find the section with id "video"
video_section = parser.css_first("section#video")

# Extract the HTML of the video section and display it
video_section_html = video_section.html if video_section else ""
display(video_section_html)

'<section id="video" aria-labelledby="video-section-label">\n    <div class="responsive-error" id="player-error" style="display: none;"><span></span></div>\n\n     \n        <div class="flowplayer fp-full fp-edgy fp-mute" data-key="$256026924867230" preload="true">\n            <video>\n                <source type="application/x-mpegurl" src="https://archive-stream.granicus.com/OnDemand/_definst_/mp4:archive/tulsa-ok/tulsa-ok_d4578b1d-786e-49f9-9230-87b182dfffaf.mp4/playlist.m3u8">\n                <track kind="captions" srclang="en" label="On" src="/videos/6399/captions.vtt">\n            </video>\n        </div>\n        <div id="granicus-controls" style="display: none;">\n                            <div class="seek-backward">\n                    <i class="fa fa-backward" aria-hidden="true"></i>\n                </div>\n\n                <div class="seek-forward">\n                    <i class="fa fa-forward" aria-hidden="true"></i>\n                </div>\n            \n         

In [27]:
# Cell to retrieve download URL and create a simple player
import asyncio
import os
from IPython.display import HTML, display
from src.granicus import get_video_player

# URL for the Granicus player page
granicus_url = "https://tulsa-ok.granicus.com/MediaPlayer.php?view_id=4&clip_id=6399"
player = await get_video_player(granicus_url)

player_html = f"""
<div style="max-width: 640px; margin: 20px auto; border: 1px solid #ddd; padding: 10px; background: #f5f5f5;">
    <h3 style="margin-top: 0; color: #333;">Direct Video Player</h3>
    
    <video id="granicus-video" controls width="100%" preload="metadata" style="background: #000;">
    <source src="{player.download_url}" type="video/mp4">
    Your browser does not support the video tag.
    </video>
    
    <div style="margin-top: 10px; font-size: 12px; color: #666;">
    <p>Using direct MP4 download URL instead of streaming URL for better playback in notebook.</p>
    </div>
</div>
"""

display(HTML(player_html))



In [44]:
# Cell to retrieve download URL and create a simple player
import asyncio
import os

from IPython.display import HTML, display
from src.granicus import get_video_player
import os
import base64
from pathlib import Path
from IPython.display import HTML, display

# Get the full absolute path to the VTT file for better compatibility
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
vtt_file = os.path.join(notebook_dir, "subtitles_transcript.vtt")

with open(vtt_file, "r") as f:
    vtt_content = f.read()

# Print first few lines for verification
# print(f"VTT file found. First 3 lines:\n{vtt_content.split('\n')[:3]}")

# Encode the VTT content as a data URL
vtt_data_url = f"data:text/vtt;charset=utf-8;base64,{base64.b64encode(vtt_content.encode('utf-8')).decode('ascii')}"

# Create player HTML with embedded VTT content
player_html = f"""
<div style="max-width: 640px; margin: 20px auto; border: 1px solid #ddd; padding: 10px; background: #f5f5f5;">
    <h3 style="margin-top: 0; color: #333;">Video Player with Embedded VTT</h3>
    
    <video id="video-with-data-vtt" controls width="100%" preload="metadata" style="background: #000;">
    <source src="{player.download_url}" type="video/mp4">
    <track kind="subtitles" srclang="en" label="English" src="{vtt_data_url}" default>
    Your browser does not support the video tag.
    </video>
    
    <div style="margin-top: 10px; font-size: 12px; color: #666;">
    <p>Using data URL for VTT content</p>
    </div>
</div>
"""

display(HTML(player_html))

In [48]:
# Cell to create a player with embedded VTT content
import os
from pathlib import Path
from IPython.display import HTML, display

# Get absolute path to the VTT file
notebook_dir = Path(os.getcwd())
vtt_file_path = notebook_dir / "subtitles_transcript.vtt"

try:
    # Read the VTT file and normalize line endings
    with open(vtt_file_path, "r", encoding="utf-8") as f:
        vtt_content = f.read()
    
    # Normalize line endings to LF (important for VTT)
    vtt_content = vtt_content.replace('\r\n', '\n')
    
    # Print first few lines for verification
    # print(f"First 3 lines of VTT:\n{vtt_content.split('\n')[:3]}")
    
    # Create embedded VTT content using Denilson Sá Maia's approach
    # from https://stackoverflow.com/questions/23553350/viewing-html5-video-with-captions-offline
    player_html = f"""
    <div style="max-width: 640px; margin: 20px auto; border: 1px solid #ddd; padding: 10px; background: #f5f5f5;">
        <h3 style="margin-top: 0; color: #333;">Video Player with JavaScript-Parsed VTT</h3>
        
        <video id="video-with-js-vtt" controls width="100%" preload="metadata" style="background: #000;">
            <source src="{player.download_url}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
        
        <script type="text/vtt" id="subtitle" data-label="English" data-lang="en">
{vtt_content}
        </script>
        
        <script>
        function parse_timestamp(s) {{
            var match = s.match(/^(?:([0-9]{{2,}}):)?([0-5][0-9]):([0-5][0-9][.,][0-9]{{0,3}})/);
            if (match == null) {{
                throw 'Invalid timestamp format: ' + s;
            }}
            var hours = parseInt(match[1] || "0", 10);
            var minutes = parseInt(match[2], 10);
            var seconds = parseFloat(match[3].replace(',', '.'));
            return seconds + 60 * minutes + 60 * 60 * hours;
        }}

        function quick_and_dirty_vtt_or_srt_parser(vtt) {{
            var lines = vtt.trim().replace('\\r\\n', '\\n').split(/[\\r\\n]/).map(function(line) {{
                return line.trim();
            }});
            var cues = [];
            var start = null;
            var end = null;
            var payload = null;
            for (var i = 0; i < lines.length; i++) {{
                if (lines[i].indexOf('-->') >= 0) {{
                    var splitted = lines[i].split(/[ \\t]+-->[ \\t]+/);
                    if (splitted.length != 2) {{
                        throw 'Error when splitting "-->": ' + lines[i];
                    }}
                    start = parse_timestamp(splitted[0]);
                    end = parse_timestamp(splitted[1]);
                }} else if (lines[i] == '') {{
                    if (start && end) {{
                        var cue = new VTTCue(start, end, payload);
                        cues.push(cue);
                        start = null;
                        end = null;
                        payload = null;
                    }}
                }} else if(start && end) {{
                    if (payload == null) {{
                        payload = lines[i];
                    }} else {{
                        payload += '\\n' + lines[i];
                    }}
                }}
            }}
            return cues;
        }}

        function init() {{
            var video = document.querySelector('#video-with-js-vtt');
            var subtitle = document.getElementById('subtitle');
            var track = video.addTextTrack('subtitles', subtitle.dataset.label, subtitle.dataset.lang);
            track.mode = "showing";
            quick_and_dirty_vtt_or_srt_parser(subtitle.innerHTML).map(function(cue) {{
                track.addCue(cue);
            }});
        }}
        
        // Run the initialization after a short delay to ensure video is loaded
        setTimeout(init, 500);
        </script>
    </div>
    """
    
    display(HTML(player_html))
    
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")